In [4]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import inspect

np.random.seed(1)

<img src="img/cnn.png">

<img src="img/cnn2.png">

## Zero padding

In [6]:
inspect.signature(np.pad)

<Signature (array, pad_width, mode, **kwargs)>

In [7]:
def zero_pad(X, pad):
    """
    X -- array of shape (m, n_H, n_W, n_C) representing m images
    pad -- int, padding width
    """
    X_pad = np.pad(X, ((0, 0), (pad, pad), (pad, pad), (0, 0)), "constant", constant_values = 0)
    return X_pad

In [8]:
x = np.random.randn(4, 3, 3, 2)
x_pad = zero_pad(x, 2)
print("x_pad.shape = ", x_pad.shape)

x_pad.shape =  (4, 7, 7, 2)


## Single step of convolution

In [12]:
def conv_single_step(a_slice_prev, W, b):
    """
    a_slice_prev -- slice of input data of shape (f, f, n_C_prev)
    W -- the filter, of shape (f, f, n_C_prev)
    b -- bias, of shape (1, 1, 1)
    """
    s = a_slice_prev * W
    Z = np.sum(s) + np.asscalar(b)
    return Z

In [13]:
np.random.seed(1)
a_slice_prev = np.random.randn(4, 4, 3)
W = np.random.randn(4, 4, 3)
b = np.random.randn(1, 1, 1)

Z = conv_single_step(a_slice_prev, W, b)
print("Z = ", Z)

Z =  -6.99908945068


## CNN - forward pass

In [24]:
def conv_forward(A_prev, W, b, hparameters):
    """
    A_prev -- activations from previous layer, of shape (m, n_H_prev, n_W_prev, n_C_prev)
    W -- filters, of shape (f, f, n_C_prev, n_C), n_C is the number of filters
    b -- biases, of shape (1, 1, 1, n_C)
    hparameters -- dict containing "stride" and "pad"
    
    Returns:
    Z -- of shape (m, n_H, n_W, n_C)
    """
    stride = hparameters["stride"]
    pad = hparameters["pad"]
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    (f, f, n_C_prev, n_C) = W.shape
    
    n_H = np.floor((n_H_prev + 2 * pad - f) / stride + 1).astype(np.int)
    n_W = np.floor((n_W_prev + 2 * pad - f) / stride + 1).astype(np.int)
    
    A_prev_pad = zero_pad(A_prev, pad)
    Z = np.zeros((m, n_H, n_W, n_C))
    for i in range(m):
        for h in range(n_H):
            for w in range(n_W):
                for c in range(n_C):
                    Z[i, h, w, c] = conv_single_step(A_prev_pad[
                        i, (h*stride):(h*stride + f), (w*stride):(w*stride + f), :],
                                                    W[:, :, :, c], b[:, :, :, c])
                    # Apply activation
                    # A[i, h, w, c] = activation(Z[i, h, w, c])
    cache = (A_prev, W, b, hparameters)
    return Z, cache

In [36]:
np.random.seed(1)
A_prev = np.random.randn(10, 4, 4, 3)
W = np.random.randn(2, 2, 3, 8)
b = np.random.randn(1, 1, 1, 8)
hparameters = {"pad" : 2, "stride": 2}

Z, cache_conv = conv_forward(A_prev, W, b, hparameters)
print("Z's mean = ", np.mean(Z))

Z's mean =  0.0489952035289


## Pooling layer

In [26]:
def pool_forward(A_prev, hparameters, mode = "max"):
    """
    A_prev -- of shape (m, n_H_prev, n_W_prev, n_C_prev)
    hparameters -- dict containing "f" and "stride"
    mode -- "max" or "average"
    
    Returns:
    A -- of shape (m, n_H, n_W, n_C)
    """
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    f = hparameters["f"]
    stride = hparameters["stride"]
    n_H = np.floor((n_H_prev - f) / stride + 1).astype(int)
    n_W = np.floor((n_W_prev - f) / stride + 1).astype(int)
    n_C = n_C_prev
    
    A = np.zeros((m, n_H, n_W, n_C))
    for i in range(m):
        for h in range(n_H):
            for w in range(n_W):
                for c in range(n_C):
                    if mode == "max":
                        A[i, h, w, c] = np.max(A_prev[i, (h*stride):(h*stride + f), (w*stride):(w*stride + f), c])
                    elif mode == "average":
                        A[i, h, w, c] = np.mean(A_prev[i, (h*stride):(h*stride + f), (w*stride):(w*stride + f), c])
    cache = (A_prev, hparameters)
    
    return A, cache

In [37]:
np.random.seed(1)
A_prev = np.random.randn(2, 4, 4, 3)
hparameters = {"f" : 3, "stride": 2}

A, cache_pool = pool_forward(A_prev, hparameters)
print("A = ", A)

A =  [[[[ 1.74481176  0.86540763  1.13376944]]]


 [[[ 1.13162939  1.51981682  2.18557541]]]]


## Backpropagation

In modern deep learning frameworks, you only have to implement the forward pass, and the framework takes care of the backward pass, so most deep learning engineers don't need to bother with the details of the backward pass. The backward pass for convolutional networks is complicated. If you wish however, you can work through this optional portion of the notebook to get a sense of what backprop in a convolutional network looks like. 

When in an earlier course you implemented a simple (fully connected) neural network, you used backpropagation to compute the derivatives with respect to the cost to update the parameters. Similarly, in convolutional neural networks you can to calculate the derivatives with respect to the cost in order to update the parameters. The backprop equations are not trivial and we did not derive them in lecture, but we briefly presented them below.

### Convolutional layer backward pass 

Let's start by implementing the backward pass for a CONV layer. 

#### Computing dA:
This is the formula for computing $dA$ with respect to the cost for a certain filter $W_c$ and a given training example:

$$ dA += \sum _{h=0} ^{n_H} \sum_{w=0} ^{n_W} W_c \times dZ_{hw} \tag{1}$$

Where $W_c$ is a filter and $dZ_{hw}$ is a scalar corresponding to the gradient of the cost with respect to the output of the conv layer Z at the hth row and wth column (corresponding to the dot product taken at the ith stride left and jth stride down). Note that at each time, we multiply the the same filter $W_c$ by a different dZ when updating dA. We do so mainly because when computing the forward propagation, each filter is dotted and summed by a different a_slice. Therefore when computing the backprop for dA, we are just adding the gradients of all the a_slices. 

In code, inside the appropriate for-loops, this formula translates into:
```python
da_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :] += W[:,:,:,c] * dZ[i, h, w, c]
```

#### Computing dW:
This is the formula for computing $dW_c$ ($dW_c$ is the derivative of one filter) with respect to the loss:

$$ dW_c  += \sum _{h=0} ^{n_H} \sum_{w=0} ^ {n_W} a_{slice} \times dZ_{hw}  \tag{2}$$

Where $a_{slice}$ corresponds to the slice which was used to generate the acitivation $Z_{ij}$. Hence, this ends up giving us the gradient for $W$ with respect to that slice. Since it is the same $W$, we will just add up all such gradients to get $dW$. 

In code, inside the appropriate for-loops, this formula translates into:
```python
dW[:,:,:,c] += a_slice * dZ[i, h, w, c]
```

#### Computing db:

This is the formula for computing $db$ with respect to the cost for a certain filter $W_c$:

$$ db = \sum_h \sum_w dZ_{hw} \tag{3}$$

As you have previously seen in basic neural networks, db is computed by summing $dZ$. In this case, you are just summing over all the gradients of the conv output (Z) with respect to the cost. 

In code, inside the appropriate for-loops, this formula translates into:
```python
db[:,:,:,c] += dZ[i, h, w, c]
```

**Exercise**: Implement the `conv_backward` function below. You should sum over all the training examples, filters, heights, and widths. You should then compute the derivatives using formulas 1, 2 and 3 above. 

In [40]:
def conv_backward(dZ, cache):
    """
    dZ -- of shape (m, n_H, n_W, n_C)
    cache -- 
    
    Returns:
    dA_prev -- of shape (m, n_H_prev, n_W_prev, n_C_prev)
    dW -- of shape (f, f, n_C_prev, n_C)
    db -- of shape (1, 1, 1, n_C)
    """
    (A_prev, W, b, hparameters) = cache
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    (f, f, n_C_prev, n_C) = W.shape
    stride = hparameters["stride"]
    pad = hparameters["pad"]
    
    (m, n_H, n_W, n_C) = dZ.shape
    
    dA_prev = np.zeros(A_prev.shape)
    dW = np.zeros(W.shape)
    db = np.zeros(b.shape)
    
    A_prev_pad = zero_pad(A_prev, pad)
    dA_prev_pad = zero_pad(dA_prev, pad)
    
    for i in range(m):
        for h in range(n_H):
            for w in range(n_W):
                for c in range(n_C):
                    dA_prev_pad[i, (h*stride):(h*stride + f), (w*stride):(w*stride + f), :] += \
                        W[:, :, :, c] * dZ[i, h, w, c]
                    dW[:, :, :, c] += A_prev_pad[i, (h*stride):(h*stride + f), (w*stride):(w*stride + f), :] * dZ[i, h, w, c]
                    db[:, :, :, c] += dZ[i, h, w, c]
        
        dA_prev[i, :, :, :] = dA_prev_pad[i, pad:-pad, pad:-pad, :]
    return dA_prev, dW, db
    

In [41]:
np.random.seed(1)
dA, dW, db = conv_backward(Z, cache_conv)
print("dA_mean = ", np.mean(dA))
print("dW_mean = ", np.mean(dW))
print("db_mean = ", np.mean(db))

dA_mean =  1.45243777754
dW_mean =  1.72699145831
db_mean =  7.83923256462


## Pooling layer - backward pass

Next, let's implement the backward pass for the pooling layer, starting with the MAX-POOL layer. Even though a pooling layer has no parameters for backprop to update, you still need to backpropagation the gradient through the pooling layer in order to compute gradients for layers that came before the pooling layer. 

### Max pooling - backward pass  

Before jumping into the backpropagation of the pooling layer, you are going to build a helper function called `create_mask_from_window()` which does the following: 

$$ X = \begin{bmatrix}
1 && 3 \\
4 && 2
\end{bmatrix} \quad \rightarrow  \quad M =\begin{bmatrix}
0 && 0 \\
1 && 0
\end{bmatrix}\tag{4}$$

As you can see, this function creates a "mask" matrix which keeps track of where the maximum of the matrix is. True (1) indicates the position of the maximum in X, the other entries are False (0). You'll see later that the backward pass for average pooling will be similar to this but using a different mask.  

**Exercise**: Implement `create_mask_from_window()`. This function will be helpful for pooling backward. 
Hints:
- [np.max()]() may be helpful. It computes the maximum of an array.
- If you have a matrix X and a scalar x: `A = (X == x)` will return a matrix A of the same size as X such that:
```
A[i,j] = True if X[i,j] = x
A[i,j] = False if X[i,j] != x
```
- Here, you don't need to consider cases where there are several maxima in a matrix.

In [42]:
def create_mask_from_window(x):
    """
    x -- of shape (f, f)
    
    Returns:
    mask -- same shape, with '1' in the maximum position, '0' in others
    """
    mask = (x == np.max(x)).astype(np.int)
    return mask

In [45]:
np.random.seed(1)
x = np.random.randn(2, 3)
mask = create_mask_from_window(x)
print("mask = ", mask)

mask =  [[1 0 0]
 [0 0 0]]


Why do we keep track of the position of the max? It's because this is the input value that ultimately influenced the output, and therefore the cost. Backprop is computing gradients with respect to the cost, so anything that influences the ultimate cost should have a non-zero gradient. So, backprop will "propagate" the gradient back to this particular input value that had influenced the cost.

### Average pooling - backward pass 

In max pooling, for each input window, all the "influence" on the output came from a single input value--the max. In average pooling, every element of the input window has equal influence on the output. So to implement backprop, you will now implement a helper function that reflects this.

For example if we did average pooling in the forward pass using a 2x2 filter, then the mask you'll use for the backward pass will look like: 
$$ dZ = 1 \quad \rightarrow  \quad dZ =\begin{bmatrix}
1/4 && 1/4 \\
1/4 && 1/4
\end{bmatrix}\tag{5}$$

This implies that each position in the $dZ$ matrix contributes equally to output because in the forward pass, we took an average. 

**Exercise**: Implement the function below to equally distribute a value dz through a matrix of dimension shape. [Hint](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ones.html)

In [49]:
def distribute_value(dz, shape):
    """
    dz -- scalar
    shape -- (n_H, n_W)
    
    Returns:
    a -- (n_H, n_W)
    """
    (n_H, n_W) = shape
    a = np.ones((n_H, n_W)) * dz / (n_H * n_W)
    return a

In [50]:
a = distribute_value(2, (2, 2))
print("a = ", a)

a =  [[ 0.5  0.5]
 [ 0.5  0.5]]


### Putting it together: Pooling backward 

You now have everything you need to compute backward propagation on a pooling layer.

**Exercise**: Implement the `pool_backward` function in both modes (`"max"` and `"average"`). You will once again use 4 for-loops (iterating over training examples, height, width, and channels). You should use an `if/elif` statement to see if the mode is equal to `'max'` or `'average'`. If it is equal to 'average' you should use the `distribute_value()` function you implemented above to create a matrix of the same shape as `a_slice`. Otherwise, the mode is equal to '`max`', and you will create a mask with `create_mask_from_window()` and multiply it by the corresponding value of dZ.

In [54]:
def pool_backward(dA, cache, mode = "max"):
    """
    dA -- gradient of cost wrt output of the pooling layer, of shape (m, n_H, n_W, n_C)
    
    Returns:
    dA_prev
    """
    (A_prev, hparameters) = cache
    stride = hparameters["stride"]
    f = hparameters["f"]
    
    (m, n_H, n_W, n_C) = dA.shape
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    
    dA_prev = np.zeros((m, n_H_prev, n_W_prev, n_C_prev))
    for i in range(m):
        for h in range(n_H):
            for w in range(n_W):
                for c in range(n_C):
                    if mode == "max":
                        mask = create_mask_from_window(A_prev[i, (h*stride):(h*stride + f), (w*stride):(w*stride + f), c])
                        dA_prev[i, (h*stride):(h*stride + f), (w*stride):(w*stride + f), c] += \
                            np.ones(mask.shape) * dA[i, h, w, c] * mask
                    elif mode == "average":
                        dA_prev[i, (h*stride):(h*stride + f), (w*stride):(w*stride + f), c] += \
                            distribute_value(dA[i, h, w, c], (f, f))
    return dA_prev

In [55]:
np.random.seed(1)
A_prev = np.random.randn(5, 5, 3, 2)
hparameters = {"stride" : 1, "f": 2}
A, cache = pool_forward(A_prev, hparameters)
dA = np.random.randn(5, 4, 2, 2)

dA_prev = pool_backward(dA, cache, mode = "max")
print("mode = max")
print('mean of dA = ', np.mean(dA))
print('dA_prev[1,1] = ', dA_prev[1,1])  
print()
dA_prev = pool_backward(dA, cache, mode = "average")
print("mode = average")
print('mean of dA = ', np.mean(dA))
print('dA_prev[1,1] = ', dA_prev[1,1]) 

mode = max
mean of dA =  0.145713902729
dA_prev[1,1] =  [[ 0.          0.        ]
 [ 5.05844394 -1.68282702]
 [ 0.          0.        ]]

mode = average
mean of dA =  0.145713902729
dA_prev[1,1] =  [[ 0.08485462  0.2787552 ]
 [ 1.26461098 -0.25749373]
 [ 1.17975636 -0.53624893]]


### All above implementations are correct!!!